In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pykrige.kriging_tools as kt


In [4]:
df_stations = pd.read_csv('Air_quality_data/Stazioni_qualit__dell_aria_20240430.csv')
df_air_quality1 = pd.read_csv('Air_quality_data/Dati_sensori_aria_2010-2017_20240430.csv', low_memory=False)
df_air_quality2 = pd.read_csv('Air_quality_data/Dati_sensori_aria_dal_2018_20240430.csv', low_memory=False)

In [5]:
# Add the two dataframes together by row
df_air_quality = pd.concat([df_air_quality1, df_air_quality2], axis=0)

In [6]:
milan_municipalities = [
    'Abbiategrasso', 'Albairate', 'Arconate', 'Arese', 'Arluno', 'Assago',
    'Baranzate', 'Bareggio', 'Basiano', 'Basiglio', 'Bellinzago Lombardo', 'Bernate Ticino', 'Besate', 'Binasco', 'Boffalora Sopra Ticino', 'Bollate', 'Bresso', 'Bubbiano', 'Buccinasco', 'Buscate', 'Bussero', 'Busto Garolfo',
    'Calvignasco', 'Cambiago', 'Canegrate', 'Carpiano', 'Carugate', 'Casarile', 'Casorezzo', 'Cassano d\'Adda', 'Cassina de\' Pecchi', 'Cassinetta di Lugagnano', 'Castano Primo', 'Cernusco sul Naviglio', 'Cerro Maggiore', 'Cerro al Lambro', 'Cesano Boscone', 'Cesate', 'Cinisello Balsamo', 'Cisliano', 'Cologno Monzese', 'Colturano', 'Corbetta', 'Cormano', 'Cornaredo', 'Corsico', 'Cuggiono', 'Cusago', 'Cusano Milanino',
    'Dairago', 'Dresano',
    'Gaggiano', 'Garbagnate Milanese', 'Gessate', 'Gorgonzola', 'Grezzago', 'Gudo Visconti',
    'Inveruno', 'Inzago',
    'Lacchiarella', 'Lainate', 'Legnano', 'Liscate', 'Locate di Triulzi',
    'Magenta', 'Magnago', 'Marcallo con Casone', 'Masate', 'Mediglia', 'Melegnano', 'Melzo', 'Mesero', 'Milano', 'Morimondo', 'Motta Visconti',
    'Nerviano', 'Nosate', 'Novate Milanese', 'Noviglio',
    'Opera', 'Ossona', 'Ozzero',
    'Paderno Dugnano', 'Pantigliate', 'Parabiago', 'Paullo', 'Pero', 'Peschiera Borromeo', 'Pessano con Bornago', 'Pieve Emanuele', 'Pioltello', 'Pogliano Milanese', 'Pozzo D\'Adda', 'Pozzuolo Martesana', 'Pregnana Milanese',
    'Rescaldina', 'Rho', 'Robecchetto con Induno', 'Robecco sul Naviglio', 'Rodano', 'Rosate', 'Rozzano',
    'San Colombano al Lambro', 'San Donato Milanese', 'San Giorgio su Legnano', 'San Giuliano Milanese', 'San Vittore Olona', 'San Zenone al Lambro', 'Santo Stefano Ticino', 'Sedriano', 'Segrate', 'Senago', 'Sesto San Giovanni', 'Settala', 'Settimo Milanese', 'Solaro',
    'Trezzano Rosa', 'Trezzano sul Naviglio', 'Trezzo sull\'Adda', 'Tribiano', 'Truccazzano', 'Turbigo',
    'Vanzaghello', 'Vanzago', 'Vaprio d\'Adda', 'Vermezzo con Zelo', 'Vernate', 'Vignate', 'Villa Cortese', 'Vimodrone', 'Vittuone', 'Vizzolo Predabissi',
    'Zibido San Giacomo'
]

In [7]:
## Print the number of municipalities in Milan
print(len(milan_municipalities))

133


In [8]:
## Make a list of municipalities in Milan that are not in the dataset
not_in_dataset = [municipality for municipality in milan_municipalities if municipality not in df_stations['Comune'].unique()]
print(not_in_dataset)

['Albairate', 'Arluno', 'Assago', 'Baranzate', 'Bareggio', 'Basiano', 'Basiglio', 'Bellinzago Lombardo', 'Bernate Ticino', 'Besate', 'Binasco', 'Boffalora Sopra Ticino', 'Bollate', 'Bresso', 'Bubbiano', 'Buccinasco', 'Buscate', 'Bussero', 'Busto Garolfo', 'Calvignasco', 'Cambiago', 'Canegrate', 'Carpiano', 'Carugate', 'Casarile', 'Casorezzo', "Cassina de' Pecchi", 'Cassinetta di Lugagnano', 'Cernusco sul Naviglio', 'Cerro Maggiore', 'Cerro al Lambro', 'Cesano Boscone', 'Cesate', 'Cisliano', 'Cologno Monzese', 'Colturano', 'Corbetta', 'Cornaredo', 'Cusago', 'Cusano Milanino', 'Dairago', 'Dresano', 'Gaggiano', 'Gessate', 'Gorgonzola', 'Grezzago', 'Gudo Visconti', 'Inveruno', 'Liscate', 'Locate di Triulzi', 'Magnago', 'Marcallo con Casone', 'Masate', 'Mediglia', 'Melzo', 'Mesero', 'Morimondo', 'Nerviano', 'Nosate', 'Novate Milanese', 'Noviglio', 'Opera', 'Ossona', 'Ozzero', 'Paderno Dugnano', 'Pantigliate', 'Parabiago', 'Paullo', 'Peschiera Borromeo', 'Pessano con Bornago', 'Pieve Emanuel

In [9]:
## Drop rows with municipalities not in the list of milan_municipalities
df_milano_stations = df_stations[df_stations['Comune'].isin(milan_municipalities)]
print(df_milano_stations['Comune'].unique())

['Inzago' 'Pero' 'Sesto San Giovanni' 'Lacchiarella' 'Cormano' 'Corsico'
 'Milano' "Cassano d'Adda" 'Cuggiono' "Trezzo sull'Adda"
 'Cinisello Balsamo' 'Legnano' 'Motta Visconti' 'Magenta' 'Turbigo'
 'Robecchetto con Induno' 'Arconate' 'Settimo Milanese' 'Pioltello' 'Rho'
 'San Giuliano Milanese' 'Arese' 'Abbiategrasso' 'Melegnano' 'Lainate'
 'Garbagnate Milanese' 'Tribiano' 'Castano Primo' 'Truccazzano']


In [10]:
## Print the number of municipalities in province of Milan that have air quality data
print(len(df_milano_stations['Comune'].unique()))

29


In [ ]:
## Count the number of rows for each unique value in 'NomeTipoSensore' column
print(df_milano_stations['NomeTipoSensore'].value_counts())

In [ ]:
## Get  the unique values of 'UnitaMisura' column
print(df_milano_stations['UnitaMisura'].unique())


In [11]:
## Get the list IdSensore for each unique value in 'NomeTipoSensore' column
idSensor_grouped_by_type = df_milano_stations.groupby('NomeTipoSensore')['IdSensore'].unique()
print(idSensor_grouped_by_type)

NomeTipoSensore
Ammoniaca                                          [20020, 20484, 10269, 20525]
Arsenico                                                  [12608, 12638, 12625]
Benzene                            [17127, 6057, 6037, 9998, 6062, 6059, 17126]
Benzo(a)pirene                                            [12641, 12611, 12629]
Biossido di Azoto             [5507, 5513, 5531, 5550, 5520, 5549, 5519, 554...
Biossido di Zolfo             [5620, 5621, 5618, 5617, 5627, 5643, 5642, 563...
BlackCarbon                                 [20455, 29001, 20004, 20465, 20005]
Cadmio                                                    [12626, 12609, 12639]
Monossido di Carbonio         [5814, 5820, 5819, 5835, 5831, 5813, 5834, 581...
Nikel                                                     [12624, 12637, 12607]
Ossidi di Azoto               [6342, 6318, 6338, 6350, 6360, 6346, 6276, 629...
Ozono                         [5712, 5714, 5710, 5713, 5724, 5725, 10350, 57...
PM10 (SM2005)           

In [12]:
## Print the number of rows in the air quality dataframe for each unique value in 'IdSensore' column
print(df_air_quality['idSensore'].value_counts())

6665     122511
5721     122428
5548     122419
6419     122390
5749     122381
          ...  
17083       248
6922        105
5769        105
6943          3
6942          3
Name: idSensore, Length: 924, dtype: int64


In [13]:
## Remove column 'idOperatore' from the air quality dataframe
df_air_quality = df_air_quality.drop('idOperatore', axis=1)
df_air_quality.head()

,idSensore,Data,Valore,Stato
0,5504,01/01/2010 01:00:00,70.7,VA
1,5504,01/01/2010 02:00:00,73.0,VA
2,5504,01/01/2010 03:00:00,69.0,VA
3,5504,01/01/2010 04:00:00,66.0,VA
4,5504,01/01/2010 05:00:00,68.8,VA


In [14]:
## Rename the column 'idSensore' to 'IdSensore' in the air quality dataframe
df_air_quality = df_air_quality.rename(columns={'idSensore': 'IdSensore'})
df_air_quality.head()

,IdSensore,Data,Valore,Stato
0,5504,01/01/2010 01:00:00,70.7,VA
1,5504,01/01/2010 02:00:00,73.0,VA
2,5504,01/01/2010 03:00:00,69.0,VA
3,5504,01/01/2010 04:00:00,66.0,VA
4,5504,01/01/2010 05:00:00,68.8,VA


In [15]:
## Add 'UnitaMisura' column to the air quality dataframe based on the 'IdSensore' column
df_air_quality = df_air_quality.merge(df_milano_stations[['IdSensore', 'UnitaMisura']], on='IdSensore')
df_air_quality.head()

,IdSensore,Data,Valore,Stato,UnitaMisura
0,5504,01/01/2010 01:00:00,70.7,VA,µg/m³
1,5504,01/01/2010 02:00:00,73.0,VA,µg/m³
2,5504,01/01/2010 03:00:00,69.0,VA,µg/m³
3,5504,01/01/2010 04:00:00,66.0,VA,µg/m³
4,5504,01/01/2010 05:00:00,68.8,VA,µg/m³


In [16]:
## Append the column 'NomeTipoSensore' to the air quality dataframe based on the 'IdSensore' column
df_air_quality = df_air_quality.merge(df_milano_stations[['IdSensore', 'NomeTipoSensore']], on='IdSensore')
df_air_quality.head()

,IdSensore,Data,Valore,Stato,UnitaMisura,NomeTipoSensore
0,5504,01/01/2010 01:00:00,70.7,VA,µg/m³,Biossido di Azoto
1,5504,01/01/2010 02:00:00,73.0,VA,µg/m³,Biossido di Azoto
2,5504,01/01/2010 03:00:00,69.0,VA,µg/m³,Biossido di Azoto
3,5504,01/01/2010 04:00:00,66.0,VA,µg/m³,Biossido di Azoto
4,5504,01/01/2010 05:00:00,68.8,VA,µg/m³,Biossido di Azoto


In [ ]:
## Get the unique values of 'UnitaMisura'  for each unique value of 'NomeTipoSensore' in the air quality dataframe
unita_misura_grouped_by_type = df_air_quality.groupby('NomeTipoSensore')['UnitaMisura'].unique()
print(unita_misura_grouped_by_type)

In [ ]:
## Remove rows where 'NomeTipoSensore' has the value 'Cadmio', 'Benzo(a)pirene', 'Nikel', 'Piombo', 'Arsenico', 'Ammoniaca', 'BlackCarbon', 'Particelle sospese PM2.5', 'Benzene'
df_air_quality = df_air_quality[~df_air_quality['NomeTipoSensore'].isin(['Cadmio', 'Benzo(a)pirene', 'Nikel', 'Piombo', 'Arsenico', 'Ammoniaca', 'BlackCarbon', 'Particelle sospese PM2.5', 'Benzene'])]
df_air_quality.head()

In [ ]:
## Count how many rows for each unique values of column 'Valore' that are less than 0 in the air quality dataframe
print(df_air_quality['Valore'][df_air_quality['Valore'] < 0].value_counts())

In [ ]:
## Convert the column 'Date' to datetime
df_air_quality['Data'] = pd.to_datetime(df_air_quality['Data'], dayfirst=True)
df_air_quality.head()

In [ ]:
## Add columns 'Year', 'Month', 'Day', 'Hour' to the air quality dataframe based on the 'Data' column
df_air_quality['Year'] = df_air_quality['Data'].dt.year
df_air_quality['Month'] = df_air_quality['Data'].dt.month
df_air_quality['Day'] = df_air_quality['Data'].dt.day
df_air_quality['Hour'] = df_air_quality['Data'].dt.hour
df_air_quality.head()

In [ ]:
## Append the column 'Comune' to the air quality dataframe based on the 'IdSensore' column
df_air_quality = df_air_quality.merge(df_milano_stations[['IdSensore', 'Comune']], on='IdSensore')
df_air_quality.head()

In [ ]:
## Save the air quality dataframe to a csv file 
df_air_quality.to_csv('df_air_quality_dateTime.csv', index=False)

# READ FILE

In [2]:
df_air_quality = pd.read_csv('df_air_quality_dateTime.csv')
df_air_quality.head()

C:\Users\massi\AppData\Local\Temp\ipykernel_39168\4169795793.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_air_quality = pd.read_csv('df_air_quality_dateTime.csv')


,IdSensore,Data,Valore,Stato,UnitaMisura,NomeTipoSensore,Year,Month,Day,Hour,Comune
0,5504,2010-01-01 01:00:00,70.7,VA,µg/m³,Biossido di Azoto,2010,1,1,1,Milano
1,5504,2010-01-01 02:00:00,73.0,VA,µg/m³,Biossido di Azoto,2010,1,1,2,Milano
2,5504,2010-01-01 03:00:00,69.0,VA,µg/m³,Biossido di Azoto,2010,1,1,3,Milano
3,5504,2010-01-01 04:00:00,66.0,VA,µg/m³,Biossido di Azoto,2010,1,1,4,Milano
4,5504,2010-01-01 05:00:00,68.8,VA,µg/m³,Biossido di Azoto,2010,1,1,5,Milano


In [5]:
## Count how many NaN values in the column 'Valore' in the air quality dataframe
print(df_air_quality['Valore'].isnull().sum())

0


In [6]:
## Put NaN in  rows with values less than 0 in the column 'Valore' in the air quality dataframe
df_air_quality.loc[df_air_quality['Valore'] < 0, 'Valore'] = np.nan


In [7]:
## Count how many NaN values in the column 'Valore' in the air quality dataframe
print(df_air_quality['Valore'].isnull().sum())

1676120


In [8]:
## Count how many NaN for each year in the column 'Valore' in the air quality dataframe
print(df_air_quality.groupby('Year')['Valore'].apply(lambda x: x.isnull().sum()))

Year
2010    255728
2011    296907
2012    346931
2013    299951
2014    110516
2015    117520
2016    114149
2017     32460
2018     26696
2019     18229
2020     14733
2021     15588
2022     17513
2023      9199
2024         0
Name: Valore, dtype: int64


In [9]:
## Count how many NaN for each IdSensore in the column 'Valore' in the air quality dataframe
print(df_air_quality.groupby('IdSensore')['Valore'].apply(lambda x: x.isnull().sum()))



IdSensore
5504      3546
5505     35892
5506      2163
5507      5737
5508      1601
         ...  
10354      232
10355     6393
20427     4257
20428      104
20429       28
Name: Valore, Length: 173, dtype: int64


In [10]:
## Add a column 'YearlyAverage' to the air quality dataframe that contains the average value of 'Valore' for each year for each unique IdSensore
df_air_quality['YearlyAverage'] = df_air_quality.groupby(['Year', 'IdSensore'])['Valore'].transform('mean')
df_air_quality.head()

,IdSensore,Data,Valore,Stato,UnitaMisura,NomeTipoSensore,Year,Month,Day,Hour,Comune,YearlyAverage
0,5504,2010-01-01 01:00:00,70.7,VA,µg/m³,Biossido di Azoto,2010,1,1,1,Milano,73.336213
1,5504,2010-01-01 02:00:00,73.0,VA,µg/m³,Biossido di Azoto,2010,1,1,2,Milano,73.336213
2,5504,2010-01-01 03:00:00,69.0,VA,µg/m³,Biossido di Azoto,2010,1,1,3,Milano,73.336213
3,5504,2010-01-01 04:00:00,66.0,VA,µg/m³,Biossido di Azoto,2010,1,1,4,Milano,73.336213
4,5504,2010-01-01 05:00:00,68.8,VA,µg/m³,Biossido di Azoto,2010,1,1,5,Milano,73.336213


In [11]:
y_average=df_air_quality[['Year', 'IdSensore', 'NomeTipoSensore','YearlyAverage']].drop_duplicates()
y_average.head()

,Year,IdSensore,NomeTipoSensore,YearlyAverage
0,2010,5504,Biossido di Azoto,73.336213
8757,2011,5504,Biossido di Azoto,79.395491
17243,2012,5504,Biossido di Azoto,67.325237
25613,2013,5504,Biossido di Azoto,57.484775
34340,2014,5504,Biossido di Azoto,55.732471


In [12]:
## for each IdSensore make a column for every year with its respective YearlyAverage
y_average = y_average.pivot_table(index=['IdSensore', 'NomeTipoSensore'], columns='Year', values='YearlyAverage')
y_average.head()

,Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
IdSensore,NomeTipoSensore,,,,,,,,,,,,,,,
5504,Biossido di Azoto,73.336213,79.395491,67.325237,57.484775,55.732471,75.268302,66.999654,64.388197,59.258082,57.610548,48.491295,44.348874,43.591399,43.825003,43.8
5506,Biossido di Azoto,64.051657,66.068298,67.154015,61.212845,47.973746,67.156559,51.702877,53.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5507,Biossido di Azoto,64.222032,67.438324,63.872621,54.068553,51.169834,60.002450,42.909536,55.655026,42.800700,42.268191,38.227980,42.031321,39.997747,37.594306,33.8
5508,Biossido di Azoto,46.620318,54.160047,57.405850,46.528702,34.282085,39.301586,38.114640,54.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5511,Biossido di Azoto,29.751904,32.469399,31.139307,33.215860,25.405678,24.758916,26.547022,23.375064,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
y_average_2015=y_average[y_average['Year']==2015]
y_average_2015.head()

KeyError: 'Year'

In [ ]:
y_average_2015_BioAz = y_average_2015[y_average_2015['NomeTipoSensore']=='Biossido di Azoto']
y_average_2015_BioAz.shape

In [ ]:
y_average_2015_BioAz.to_csv('y_average_2015_BioAz.csv', index=False)

In [ ]:
## Plot the average value of 'Valore' for each year for each unique IdSensore
sns.lineplot(x='Year', y='YearlyAverage', hue='NomeTipoSensore', data=df_air_quality)
plt.legend(fontsize='x-small', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
## Get the maximum value of 'Valore' for each unique NomeTipoSensore for each year in the air quality dataframe
max_values = df_air_quality.groupby(['Year', 'NomeTipoSensore'])['Valore'].max()
print(max_values)

In [ ]:
## Create a dataframe with each unique value of 'NomeTipoSensore' as index and the YearlyAverage for each year as columns
df_yearly_average = df_air_quality.pivot_table(index='NomeTipoSensore', columns='Year', values='YearlyAverage')
df_yearly_average.head()

In [ ]:
## Plot number of rows for each unique value in 'NomeTipoSensore' column per year in the air quality dataframe
sns.countplot(x='Year', hue='NomeTipoSensore', data=df_air_quality)
plt.legend(fontsize='x-small', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
## Df with YearlyAverage, max and min values of 'Valore' for each unique value of 'NomeTipoSensore', and the variance of 'Valore' for each unique value of 'NomeTipoSensore'
df_yearly_summary = df_air_quality.groupby(['Year', 'NomeTipoSensore'])['Valore'].agg(['mean', 'max', 'min', 'var'])
df_yearly_summary

In [ ]:
## Plot the max column of the df_yearly_summary dataframe
sns.lineplot(x='Year', y='max', hue='NomeTipoSensore', data=df_yearly_summary)
plt.legend(fontsize='x-small', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
## Create a dataframe with each unique value of 'NomeTipoSensore' as index and the average value of 'Valore' for all the rows that correspond to the same day as columns
df_daily_average = df_air_quality.pivot_table(index='NomeTipoSensore', columns=['Year', 'Month', 'Day'], values='Valore', aggfunc='mean')
df_daily_average.head()

In [ ]:
## Get the max value of the daily average for each unique value of 'NomeTipoSensore'
max_daily_average = df_daily_average.max(axis=1)
print(max_daily_average)



In [ ]:
## Plot the max value of the daily average for each unique value of 'NomeTipoSensore'
sns.barplot(x=max_daily_average, y=max_daily_average.index)
plt.show()

In [ ]:
## Get how many times the daily average is greater than 50 for Particelle sospese PM2.5 row for each year
df_air_quality[(df_air_quality['NomeTipoSensore'] == 'Particelle sospese PM2.5') & (df_air_quality['Valore'] > 50)].groupby('Year').size()

In [17]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
df_air_quality

,IdSensore,Data,Valore,Stato,UnitaMisura,NomeTipoSensore
0,5504,01/01/2010 01:00:00,70.7,VA,µg/m³,Biossido di Azoto
1,5504,01/01/2010 02:00:00,73.0,VA,µg/m³,Biossido di Azoto
2,5504,01/01/2010 03:00:00,69.0,VA,µg/m³,Biossido di Azoto
3,5504,01/01/2010 04:00:00,66.0,VA,µg/m³,Biossido di Azoto
4,5504,01/01/2010 05:00:00,68.8,VA,µg/m³,Biossido di Azoto
...,...,...,...,...,...,...
12343219,20529,28/12/2023 00:00:00,39.0,VA,µg/m³,Particelle sospese PM2.5
12343220,20529,29/12/2023 00:00:00,27.0,VA,µg/m³,Particelle sospese PM2.5
12343221,20529,30/12/2023 00:00:00,28.0,VA,µg/m³,Particelle sospese PM2.5
12343222,20529,31/12/2023 00:00:00,40.0,VA,µg/m³,Particelle sospese PM2.5


In [18]:
## Select only the rows where 'NomeTipoSensore' is 'PM10 (SM2005)'
df_air_quality_PM10 = df_air_quality[df_air_quality['NomeTipoSensore'] == 'PM10 (SM2005)']
df_air_quality_PM10.head()

,IdSensore,Data,Valore,Stato,UnitaMisura,NomeTipoSensore
10193898,6905,01/01/2010 00:00:00,-9999.0,NaN,µg/m³,PM10 (SM2005)
10193899,6905,02/01/2010 00:00:00,-9999.0,NaN,µg/m³,PM10 (SM2005)
10193900,6905,03/01/2010 00:00:00,-9999.0,NaN,µg/m³,PM10 (SM2005)
10193901,6905,04/01/2010 00:00:00,-9999.0,NaN,µg/m³,PM10 (SM2005)
10193902,6905,05/01/2010 00:00:00,-9999.0,NaN,µg/m³,PM10 (SM2005)
